In [ ]:
#掩码语言训练

In [ ]:
#导入相关包
from datasets import load_dataset,Dataset
from transformers import AutoTokenizer,AutoModelForMaskedLM,DataCollatorForLanguageModeling,Trainer,TrainingArguments

In [ ]:
#加载数据集
ds = Dataset.load_from_disk("./wiki_cn_filtered")

In [ ]:
ds[0]

In [ ]:
#数据处理
tokenizer = AutoTokenizer.from_pretrained("D:\Hugging Face Hub\chinese-macbert-base")
def process_fun(examples):
    return tokenizer(examples["completion"],max_length = 384,truncation =True)

In [ ]:
tokenizer_ds = ds.map(process_fun,batched=True,remove_columns=ds.column_names)
print(tokenizer_ds[0])

In [ ]:
from torch.utils.data import DataLoader
dl = DataLoader(tokenizer_ds,batch_size=2,collate_fn=DataCollatorForLanguageModeling(tokenizer,mlm=True,mlm_probability=0.15))

In [ ]:
#创建模型
model = AutoModelForMaskedLM.from_pretrained("D:\Hugging Face Hub\chinese-macbert-base")

In [ ]:
#不配置评估函数

In [ ]:
args = TrainingArguments(
    output_dir="./mask",
    per_device_train_batch_size = 2,
    logging_steps = 100,
    num_train_epochs=1
)

In [ ]:
trainer = Trainer(
    args = args,
    model = model,
    train_dataset=tokenizer_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm = True,mlm_probability=0.15)
)

In [ ]:
#训练
trainer.train()

In [ ]:
#模型推理
from transformers import pipeline
pipe = pipeline("fill-mask",model=model,tokenizer=tokenizer,device=0)

In [ ]:
pipe("西安交通[MASK]学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")